In [1]:
## Code to verify selenium is connected to the correct page. Note that a Trip Details page must be open

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

### Configure Chrome options to connect to debugger
options = Options()
options.debugger_address = "localhost:9222"  # Port matches the Chrome startup command

### Path to ChromeDriver
driver_path = r"C:/Users/Brett Alan/Documents/Python/Selenium/chromedriver.exe"

### Use Service to specify the driver
service = Service(driver_path)

### Initialize WebDriver connected to Chrome Debugger
driver = webdriver.Chrome(service=service, options=options)

### Verify connection by printing the current page title
print(driver.title)




Uber | Payments


In [9]:
def select_week_and_load_more(driver, week_start_day):
    """
    Selects a specific week and clicks 'Load More' until all trips are listed.

    Args:
        driver: Selenium WebDriver instance.
        week_start_day: Integer (Monday's date in the selected week, e.g., 16 for December 16).

    Returns:
        None
    """
    try:
        # Step 1: Switch to 'Uber | Payments' tab
        for handle in driver.window_handles:
            driver.switch_to.window(handle)
            if driver.title == "Uber | Payments":
                print("Switched to the 'Uber | Payments' tab.")
                break
        else:
            raise Exception("Correct tab not found")

        # Step 2: Locate and interact with the search box
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@aria-label='Select a date range.']"))
        )
        search_box.click()
        print("Clicked the search box.")

        # Select the specified week
        day_to_select = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//div[text()='{week_start_day}']"))
        )
        day_to_select.click()
        print(f"Selected week starting on day {week_start_day}.")

        # Step 3: Click 'Load More' until all trips are displayed
        while True:
            try:
                # Check if the 'Load More' button exists
                load_more_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, '_css-gYTGpU')]"))
                )
                if load_more_button.is_displayed():
                    load_more_button.click()
                    print("Clicked 'Load More' button.")
                else:
                    print("No more 'Load More' buttons to click.")
                    break
            except Exception:
                print("No more 'Load More' buttons or they are no longer visible.")
                break

    except Exception as e:
        print(f"Error in select_week_and_load_more: {e}")


In [10]:
# Select week of December 16 and load all trips
select_week_and_load_more(driver, 16)


Switched to the 'Uber | Payments' tab.
Clicked the search box.
Selected week starting on day 16.
No more 'Load More' buttons or they are no longer visible.


In [13]:
from selenium.webdriver.common.by import By
import csv

def extract_trip_details(driver, csv_file_path):
    """
    Extracts Trip Type, Date, and Time from the "Trip Details" page
    and appends to a CSV file.

    Args:
        driver: Selenium WebDriver instance.
        csv_file_path: Path to the CSV file for appending data.

    Returns:
        None
    """
    try:
        # Ensure we're on the "Trip Details" tab
        driver.switch_to.window(driver.window_handles[-1])  # Switch to the last tab
        print(f"Current page: {driver.title}")

        # Locate the element containing Trip Type, Date, and Time
        trip_details_element = driver.find_element(By.CLASS_NAME, "_css-gQdIWP")
        trip_details_text = trip_details_element.text
        print(f"Extracted text: {trip_details_text}")

        # Split the text to isolate Trip Type, Date, and Time
        trip_type, date, time = map(str.strip, trip_details_text.split("•"))
        print(f"Trip Type: {trip_type}, Date: {date}, Time: {time}")

        # Append extracted data to the CSV file
        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([date, time, trip_type])  # Add other columns as needed

        print("Successfully appended data to CSV.")

    except Exception as e:
        print(f"Error extracting trip details: {e}")

# Example usage
csv_file_path = r"C:\Users\Brett Alan\Documents\Data Analysis\Refresher Work\uber_data.csv"
extract_trip_details(driver, csv_file_path)


Current page: Uber | Trip Details
Extracted text: Premier • Dec 22, 2024 • 5:17 PM
$38.69
You earned more for this trip because your rider left a tip. Sweet! 🎉
Duration
16 min 10 sec
Distance
9.96 mi
SW 4th Ave, Portland, OR 97201-5501, US
NW Burton Rd, Portland, OR 97229-4326, US
3 points earned
$5.00 tip included
Thanks sent
Trip Type: Premier, Date: Dec 22, 2024, Time: 5:17 PM
$38.69
You earned more for this trip because your rider left a tip. Sweet! 🎉
Duration
16 min 10 sec
Distance
9.96 mi
SW 4th Ave, Portland, OR 97201-5501, US
NW Burton Rd, Portland, OR 97229-4326, US
3 points earned
$5.00 tip included
Thanks sent
Successfully appended data to CSV.


In [24]:
from selenium.webdriver.common.by import By

# Ensure you're on the "Trip Details" tab
driver.switch_to.window(driver.window_handles[-1])
print(f"Current page: {driver.title}")

# Locate the element containing all the text
trip_details_element = driver.find_element(By.CLASS_NAME, "_css-gFhnMl")
trip_details_text = trip_details_element.text.strip()
print(f"Extracted text:\n{trip_details_text}")

# Split the text into lines
lines = trip_details_text.split("\n")
print("Split lines:")
for i, line in enumerate(lines):
    print(f"{i}: {line}")

# Step-by-step parsing
try:
    # Parse trip type, date, and time from the first line
    trip_type, date, time = map(str.strip, lines[0].split("•"))
    print(f"Trip Type: {trip_type}, Date: {date}, Time: {time}")
except Exception as e:
    print(f"Error parsing trip type, date, and time: {e}")

try:
    # Parse duration and distance
    duration = lines[5].split(":")[1].strip()
    print(f"Duration: {duration}")
    distance = lines[6].split(":")[1].strip()
    print(f"Distance: {distance}")
except Exception as e:
    print(f"Error parsing duration and distance: {e}")

try:
    # Parse street and city/zip code
    street = lines[8].strip()
    print(f"Street: {street}")
    city_zip = lines[9].strip()
    city, zip_code = city_zip.split(",")[0], city_zip.split(",")[1].strip()
    print(f"City: {city}, Zip Code: {zip_code}")
except Exception as e:
    print(f"Error parsing street, city, and zip code: {e}")

try:
    # Parse fare and tip
    fare = lines[12].split(":")[1].strip()
    print(f"Fare: {fare}")
    tip = lines[13].split(":")[1].strip()
    print(f"Tip: {tip}")
except Exception as e:
    print(f"Error parsing fare and tip: {e}")



Current page: Uber | Trip Details
Extracted text:
Premier • Dec 22, 2024 • 5:17 PM
$38.69
Split lines:
0: Premier • Dec 22, 2024 • 5:17 PM
1: $38.69
Trip Type: Premier, Date: Dec 22, 2024, Time: 5:17 PM
Successfully appended: Dec 22, 2024, 5:17 PM, Premier


In [81]:
from selenium.webdriver.common.by import By

try:
    # Get all window handles
    handles = driver.window_handles
    print(f"Open tabs: {len(handles)}")

    # Iterate through the tabs to find the "Trip Details" tab
    for handle in handles:
        driver.switch_to.window(handle)
        if "Uber | Trip Details" in driver.title:
            print(f"Switched to tab: {driver.title}")
            break
    else:
        raise Exception("Trip Details tab not found.")

    # Verify you are on the correct tab
    print(f"Current tab title: {driver.title}")

    # Proceed with the rest of the scraping
    # Locate the dropdown and extract the text
    dropdown_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//svg[@title='Chevron down small']"))
    )
    ActionChains(driver).move_to_element(dropdown_element).click(dropdown_element).perform()
    print("Dropdown expanded successfully.")

    # Extract text
    body_element = driver.find_element(By.TAG_NAME, "body")
    page_text = body_element.text.strip()
    print(f"Extracted text:\n{page_text}")

except Exception as e:
    print(f"Error switching to Trip Details tab or extracting text: {e}")



Open tabs: 4
Switched to tab: Uber | Trip Details
Current tab title: Uber | Trip Details
Error switching to Trip Details tab or extracting text: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D1416CF5+28821]
	(No symbol) [0x00007FF6D1383880]
	(No symbol) [0x00007FF6D122578A]
	(No symbol) [0x00007FF6D12791BE]
	(No symbol) [0x00007FF6D12794AC]
	(No symbol) [0x00007FF6D12C2647]
	(No symbol) [0x00007FF6D129F33F]
	(No symbol) [0x00007FF6D12BF412]
	(No symbol) [0x00007FF6D129F0A3]
	(No symbol) [0x00007FF6D126A778]
	(No symbol) [0x00007FF6D126B8E1]
	GetHandleVerifier [0x00007FF6D174FCED+3408013]
	GetHandleVerifier [0x00007FF6D176745F+3504127]
	GetHandleVerifier [0x00007FF6D175B63D+3455453]
	GetHandleVerifier [0x00007FF6D14DBDFB+835995]
	(No symbol) [0x00007FF6D138EB9F]
	(No symbol) [0x00007FF6D138A854]
	(No symbol) [0x00007FF6D138A9ED]
	(No symbol) [0x00007FF6D137A1D9]
	BaseThreadInitThunk [0x00007FFBAEB1E8D7+23]
	RtlUserThreadStart [0x00007FFBB0AFFBCC+44]



In [85]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    # Switch to the "Trip Details" tab
    for handle in driver.window_handles:
        driver.switch_to.window(handle)
        if "Uber | Trip Details" in driver.title:
            print(f"Switched to tab: {driver.title}")
            break
    else:
        raise Exception("Trip Details tab not found.")
    
    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("Trip Details page loaded successfully.")

    # Debug to find dropdown elements
    dropdown_elements = driver.find_elements(By.XPATH, "//svg[@title='Chevron down small']")
    print(f"Dropdown elements found: {len(dropdown_elements)}")

    if dropdown_elements:
        # Scroll to the first dropdown and click
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown_elements[0])
        ActionChains(driver).move_to_element(dropdown_elements[0]).click(dropdown_elements[0]).perform()
        print("Dropdown expanded successfully.")

        # Extract all visible text
        full_page_element = driver.find_element(By.TAG_NAME, "body")
        page_text = full_page_element.text.strip()
        print(f"Extracted page text:\n{page_text}")
    else:
        raise Exception("No dropdown elements found.")

except Exception as e:
    print(f"Error interacting with Trip Details tab: {e}")



Switched to tab: Uber | Trip Details
Trip Details page loaded successfully.
Dropdown elements found: 0
Error interacting with Trip Details tab: No dropdown elements found.


In [ ]:
# ###So first maybe I should tell you the planned workflow. 
# 1. Select Week> Load more until there are no more trips to load. (After all trips have loaded, all the trips for the week will be listed.
# 2. View Details button will be displayed by each trip and we will want to click on each  view details button on each trip.
# 3. Each time we click on View details we will then be able to click on the download CSV button. Then close the tab and click on the next view details button until we have downloaded each csv for each trip.
# # 4. Then we will want to Select on the next week for the specified range.